In [3]:
import sys,os
sys.path.append("../")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer


from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import pairwise_distances  
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import normalized_mutual_info_score

## reading the change object and clustering.

In [4]:
article_name = "John_Logie_Baird"
change_object_dir =  "../data/change objects/"
content_dir = "../data/content/"
filename =  f"{article_name}_change.h5"
change_object_file = os.path.join(change_object_dir, filename)
filename = article_name + ".h5"
filepath = os.path.join(content_dir, filename)
with pd.HDFStore(filepath, 'r') as store:
    token_string_df = store.get("all_tokens")

token_string_df = token_string_df.set_index("token_id")["str"]
token_string_df[-1] = "St@rt"
token_string_df[-2] = "$nd"


if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape


change_object_dataframe["ins_length"] = change_object_dataframe["ins_tokens"].str.len()
change_object_dataframe["del_length"] = change_object_dataframe["del_tokens"].str.len()

change_object_dataframe["del_string_tokens"] = change_object_dataframe["del_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))

change_object_dataframe["ins_string_tokens"] = change_object_dataframe["ins_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))

change_object_dataframe["left_context"] = change_object_dataframe["left_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")


change_object_dataframe["right_context"] = change_object_dataframe["right_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")

change_object_dataframe["bykau_cluster"] = pd.Series(-99,index=change_object_dataframe.index)

# change_object_dataframe = change_object_dataframe[["left_context","del_string_tokens","ins_string_tokens", "right_context", 
#                                                    "ins_length", "del_length", "bykau_cluster" ]]
change_object_dataframe.head()

ins_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10.0   
                                                                             1           32.0   
                                                                             2           34.0   
                                                                             3           50.0   
                                                                             4           57.0   

                                                                                ins_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         30.0   
                                                                             1         32.0   
                                                                             2         34.0   
                                                                             3         55.0   
                                                                             4         73.0   

                                                                                left_neigh  \
from revision id to revision id timestamp           timegap           editor                 
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           9   
                                                                             1          10   
                                                                             2          11   
                                                                             3          26   
                                                                             4          32   

                                                                                right_neigh  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10   
                                                                             1           11   
                                                                             2           12   
                                                                             3           32   
                                                                             4           36   

                                                                                del_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           -1.0   
                                                                             1           -1.0   
                                                                             2           -1.0   
                                                                             3           27.0   
                                                                             4           33.0   

                                                                                del_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         -1.0   
                                                                             1         -1.0   
                                                                             2         -1.0   
                                                                             3         31.0   
                                                                             4         35.0   

                                                               

## Reduce the change object.

In [ ]:
ins_and_del = change_object_dataframe[(change_object_dataframe["ins_string_tokens"]!=()) & (change_object_dataframe["del_string_tokens"]!=())]
display(ins_and_del.shape)

### Removing change object with insert or delete token size more than five.

In [ ]:
reduced_ins_and_del = ins_and_del[~((ins_and_del["ins_length"] >5 ) | (ins_and_del["del_length"] >5) )]
reduced_ins_and_del.shape

In [ ]:
reduced_ins_and_del.reset_index(drop=True)[["ins_string_tokens", "del_string_tokens"]].sample(3)

### Removing low user support tokens

In [ ]:
bykau_change_object = reduced_ins_and_del.groupby("ins_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object.shape

### Clustering using jaccard

##### Defining jaccard similarity function.

In [5]:
r_threshold = 8
cutoff_threshold = 0.75

In [6]:
%%time
# edit_tokens = change_object_dataframe["ins_tokens"] + change_object_dataframe["del_tokens"]
left_neighbours = change_object_dataframe["left_context"].apply(lambda x: x.split(" ")[-r_threshold:])
right_neighbours = change_object_dataframe["right_context"].apply(lambda x: x.split(" ")[:r_threshold])
neighbour_tokens = left_neighbours + right_neighbours

# bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
neighbour_tokens_set = neighbour_tokens.apply(lambda x: np.unique(x))
neighbour_vec = MultiLabelBinarizer().fit_transform(neighbour_tokens_set)

db = DBSCAN(eps=cutoff_threshold, min_samples=5, metric='jaccard').fit(neighbour_vec)
change_object_dataframe.loc[change_object_dataframe.index,"bykau_cluster"]= db.labels_

CPU times: user 4min 9s, sys: 916 ms, total: 4min 10s
Wall time: 4min 9s


#### Saving change object and its clusters

In [17]:
bykau_dir =  "../data/bykau_change_object/"
filename =  f"{article_name}_without_optimization.h5"

change_object_file = os.path.join(bykau_dir, filename)
with pd.HDFStore(change_object_file, 'w') as store:
    store.put("data", change_object_dataframe["bykau_cluster"],)

### Uncomment for seeing the content of each cluster in an ipywidget User interface

In [8]:
# repers_first =  change_object_dataframe.groupby("bykau_cluster")[["left_context","del_string_tokens", "ins_string_tokens", "right_context"]].apply(lambda x: x.style.render())
# @interact( clusters_html=fixed(repers_first), group=change_object_dataframe["bykau_cluster"].unique().tolist())
# def display_clusters(clusters_html, group):
#      return display(HTML(clusters_html.loc[group]))

#### reading the annotation and comparing

In [9]:
def token_in_gap(ann, gap):
    context_gap = gap.loc[ann[['revid_ctxt', 'rev_id']]]
    clusters = context_gap.loc[ context_gap["token_id"].apply(
            lambda x: ann["token_id"] in x), ["bykau_cluster"]].values
    if clusters.size >0:
            clusters = pd.Series(clusters[0],index=["bykau_cluster",])
    else:
        clusters = pd.Series([-10], index=["bykau_cluster",])
    return clusters

In [10]:
file_name = article_name + "_FULL.csv"
annotation_dir = "../data/annotation/"
full_file_path = os.path.join(annotation_dir, file_name)
annotation_df = pd.read_csv(full_file_path)

true_labels = annotation_df[[ "nationality", "birth_place",]]
annotation_df = annotation_df[["revid_ctxt", "rev_id", "token_id",]] 


In [11]:
# insert array is always done in to revision so taking it and leaving other change object where 
ins_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id',"bykau_cluster"]].values

# delete array is always done in from revision so taking it and leaving other change object where delete does not come.
del_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id',"bykau_cluster"]].values

gap_array = np.concatenate([ins_array,del_array], axis=0)
gap_df = pd.DataFrame(gap_array,columns= ["revid_ctxt", "token_id", "rev_id","bykau_cluster"])
gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])

In [12]:
# Finding the tokens who were in the gap.
annotation_df["bykau_cluster"] = annotation_df.apply(token_in_gap, axis=1, args=(gap_df,))

In [13]:
nationality_cluster = np.zeros((true_labels.shape[0]))
nationality_cluster[true_labels["nationality"].str.strip() == "Y"] = 1

In [14]:
evaluation_score = pd.Series(index=["rand", "mutual_info",])
evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], nationality_cluster)

evaluation_score["mutual_info"] = adjusted_mutual_info_score(nationality_cluster, 
                                            annotation_df["bykau_cluster"]  )
normalized_mutual_info_score(nationality_cluster, annotation_df["bykau_cluster"])
evaluation_score

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


rand          -0.001377
mutual_info   -0.000504
dtype: float64

In [15]:
both_cluster = np.zeros((true_labels.shape[0]))
both_cluster[true_labels["nationality"].str.strip() == "Y"] = 1
both_cluster[true_labels["birth_place"].str.strip() == "Y"] = 2

In [16]:
evaluation_score = pd.Series(index=["rand", "mutual_info",])
evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], both_cluster)

evaluation_score["mutual_info"] = adjusted_mutual_info_score(both_cluster, 
                                            annotation_df["bykau_cluster"]  )

evaluation_score

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


rand          -0.000885
mutual_info   -0.000371
dtype: float64